## 1. Import library a modulů
V této buňce si musíme implementovat funkce z jednotlivých souborů **'mujbalik'**, které budeme používat

In [ ]:
from mujbalik.load_data import load_json_data, convert_json_to_csv
from mujbalik.physics import compute_accelerations
from mujbalik.simulation import run_simulation
from mujbalik.visualization import create_animation
from mujbalik.random_place import generate_random_scenario
from mujbalik.experiments import run_timestep_experiment
from mujbalik.current_position import current_positions
from mujbalik.load_data import save_scenario_to_csv

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# potřebujeme zvýšit limitu pro vykreslováné animací, aby se mam vešli všechny planety do okna
novy_limit_v_MB = 100.0

print(f"Nastavuji nový limit pro vkládání animací v Matplotlibu na {novy_limit_v_MB} MB.")
plt.rcParams['animation.embed_limit'] = novy_limit_v_MB


## 2. Načtení a konverze dat ze souboru .json
Abychom mohli simulaci spustit, potřebujeme načíst počáteční hodnoty pro všechny planety:
- jejich **pozice** (x, y souřadnice)
- jejich **rychlost** (vx, vy složky)
- jejich **hmotnosti**

Data budeme načítat z **.json** souboru (k tomu slouží funkce **'load_json_data'**).

Data nadále musíme připravit jako NumPy pole, protože to další funkce vyžadují.

In [ ]:
# načteni dat ze .json souboru
json_data = load_json_data("data/planets.json")
# převod dat .json -> .csv
convert_json_to_csv(json_data, "data/planets.csv")

## 3. Vykreslení aktuální polohy planet

Použijeme funkci **'current_positions'**, která umí vykreslit aktuální polohy planet na grafu.

Pro spuštění funkce potřebujeme data ze souboru **'planets.csv'** (který jsme si díky minulé funkci vytvořili).

In [ ]:
current_positions("data/planets.csv")

Připravíme si data pro výpočty gravitačního zrychlení a poté i pro simulace

In [ ]:
df = pd.read_csv("data/planets.csv")
positions = df[['position_x', 'position_y']].to_numpy()
velocities = df[['velocity_x', 'velocity_y']].to_numpy()
masses = df['mass'].to_numpy()
names = df['name'].tolist()

## 4. Výpočet gravitačního zrychlení
Díky téhle funkci zjistíme, jak moc a kterým směrem se planety kvůli gravitaci pohybují rychleji.

K výpočtu používáme **Newtonův gravitační zákon**. Každá dvojice planet se přitahuje silou, která závisí na jejich hmotnostech a vzdálenosti mezi nimi.

Výsledek této funkce **'compute_accelerations'** pak použijeme při simulaci.

Pokud chceme, můžeme si vytisknout výsledky gravitačního zrychlení pro kontrolu.

In [ ]:
accelerations = compute_accelerations(positions, masses)

print("Zrychlení:\n", accelerations)

## 5. Hlavní simulace

Tohle je hlavní část -> spuštění simulace pohybu planet.

Tahle funkce dostane:
- počáteční pozice planet (positions)
- jejich počáteční rychlost (velocities)
- hmotnost planet (masses)
- délku simulačního kroku (dt)
- kolik kroků máme udělat (number_of_steps)

Na konci nám vrací **historii** pozic všech planet.

In [ ]:
dt = 60 * 60 * 24  # 1 den v sekundách
numbers_of_steps = 365   # počet kroků simulace (rok = 365 dní)

history = run_simulation(positions, velocities, masses, dt, numbers_of_steps)

## 6. Animace

Když máme spočítanou celou historii pozic planet (z předchozí funkce), můžeme pomocí ní vytvořit animaci.

Pomocí historie můžeme vykreslit, jak se planety hýbou v čase a zároveň jejich trajektorie.

Animaci můžeme buď:
- ukázat rovnou v notebooku
- uložit si ji jako soubor (např. .gif)

In [ ]:
create_animation(history, names)

Pokud si chceme uložit animaci do jako **.git** soubor, musím přidat **"save_path = "nazev_souboru.git""**

In [ ]:
create_animation(history, names, save_path = "planetary_simulation.gif")

## 6. Generování různých časových délek
Funkce **'run_timestep_experiment'** vezme data planet z **.csv** souboru a počet kroků, který mu zadáme. Pak pro každý z testů spustí simulaci a vykreslí trajektorie.

Vyzkoušet různé délky časového kroku (např. hodina, den, týden, ...).

**POZOR:**
- když je **'dt'** moc velké (více jak jeden měsíc), výpočet nemusí být přesný a simulace se může začít chovat divně.

In [ ]:
run_timestep_experiment('data/planets.csv', n_steps=365)

## 7. Generování náhodných scénářů

Tato funkce generuje náhodné scénáře pro simulaci, která bude náhodně generovat počáteční podmínky planet (polohy, rychlosti a hmotnosti).

In [ ]:
# rozmení pro generování náhodného scénáře
number_of_bodies = 4  # počet planet
position_range = (-1e12, 1e12)  # pozice v osách x a y
velocity_range = (-1e4, 1e4)  # rychlost v osách x a y
mass_range = (1e20, 1e25)  # hmotnost planet

# vytvoření náhodného scénáře
names_gen, positions_gen, velocities_gen, masses_gen = generate_random_scenario(number_of_bodies,
                                                                                position_range,
                                                                                velocity_range,
                                                                                mass_range)

print("Názvy těles: \n", names_gen)
print("---------------------------------------------")
print("Počáteční pozice planet:\n", positions_gen)
print("---------------------------------------------")
print("Počáteční rychlost planet:\n", velocities_gen)
print("---------------------------------------------")
print("Počáteční hmotnost planet:\n", masses_gen)
print("---------------------------------------------")


S nově vygenerovanými daty budeme pracovat dále.

Můžeme znovu projet všechny funkce, které jsme vytvořili, a zjistit, jestli fungují i s jinými daty.

Nyní znovu zavoláme všechny funkce:
- uložení vygenerovaných dat do CSV souboru
- načtení uložených dat a zobrazení aktuálních pozic
- načtení dat ze souboru do proměnných pro další výpočty
- výpočet zrychlení způsobeného gravitační interakcí mezi planetami
- spuštění simulace pohybu planet
- vytvoření animace z průběhu simulace

In [ ]:
save_scenario_to_csv(names_gen, positions_gen, velocities_gen, masses_gen, filename = "data/planety_scenar.csv")

In [ ]:
current_positions("data/planety_scenar.csv")

In [ ]:
df = pd.read_csv("data/planety_scenar.csv")
positions = df[['position_x', 'position_y']].to_numpy()
velocities = df[['velocity_x', 'velocity_y']].to_numpy()
masses = df['mass'].to_numpy()
names = df['name'].tolist()

In [ ]:
accelerations = compute_accelerations(positions, masses)
print("Zrychlení:\n", accelerations)

In [ ]:
# parametry simulace
dt = 60 * 60 * 24  # 1 den v sekundách
numbers_of_steps = 1000  # počet kroků simulace (rok = 365 dní)

history = run_simulation(positions, velocities, masses, dt, numbers_of_steps)

In [ ]:
create_animation(history, names)

In [ ]:
run_timestep_experiment('data/planety_scenar.csv', n_steps=365)